# Agentes y Retrieval-Augmented Generation (RAG) en DSPy

Este tutorial te muestra cómo trabajar con agentes, control de salida y búsquedas semánticas (RAG) usando [DSPy](https://dspy.ai/), una librería moderna para crear sistemas de IA generativa controlando su estructura y lógica.

La Inteligencia Artificial Generativa ha revolucionado la forma en que interactuamos con sistemas computacionales, permitiendo que tareas complejas puedan ser resueltas mediante el entendimiento y razonamiento en lenguaje natural. Sin embargo, para que un gran modelo de lenguaje (LLM) sea realmente útil en aplicaciones del mundo real, suele ser necesario controlar su salida, estructurar su lógica y conectarlo con datos u operaciones externas. Aquí es donde DSPy destaca, permitiendo tomar el control de cada etapa del proceso de generación de información y agentes inteligentes.

---

## ¿Qué es un Agente?

Un **agente** es una entidad artificial capaz de percibir su entorno, tomar decisiones y actuar sobre él. Un agente es, en esencia, una pieza de software con autonomía, que puede observar información, tomar decisiones basadas en razonamiento o reglas, y ejecutar acciones concretas que modifican el estado del entorno. En IA, un agente puede razonar, consultar datos, hablar y ejecutar acciones usando habilidades predefinidas (“tools”). Esta versatilidad permite que los agentes se utilicen en una variedad amplia de aplicaciones, como asistentes personales, bots clínicos, sistemas de soporte, planeadores y más.

La noción de agente proviene de la filosofía y la inteligencia artificial clásica, en la que se considera que un sistema es capaz de tener objetivos (deseos), poseer conocimientos sobre el entorno (creencias), idear planes (intenciones), y finalmente elegir cómo actuar (acciones).


DSPy puede trabajar con modelos locales o comerciales de LLMs; aquí usaremos OpenAI. Este paso es fundamental para que el sistema pueda comprender y generar lenguaje. 

In [ ]:
from pydantic import BaseModel  # Para modelos de datos estructurados
import dspy  # Librería de agentes y control de LLMs
import os
from pprint import pprint

# Configura tu clave de API de OpenAI
os.environ["OPENAI_API_KEY"] = "tu_clave_de_api_aqui"

# Elige el modelo, puedes usar cualquier compatible con OpenAI
dspy.configure(lm=dspy.LM("openai/gpt-4.1"))

## Uso de DSPy

DSPy permite forzar que las respuestas sean estructuradas y controladas, cumpliendo tu esquema de datos. Así, es posible garantizar que la salida generada siempre pueda ser interpretada y utilizada por el resto de tu aplicación, evitando respuestas ambiguas o inconsistentes.

En este ejemplo, DSPy se encarga de transformar la pregunta en una llamada estructurada al modelo de lenguaje, y exige que la respuesta contenga únicamente la información necesaria y no datos innecesarios.

In [2]:
qa = dspy.Predict('question: str -> response: str') # Define un modelo que recibe una pregunta y devuelve una respuesta
response = qa(question="Para qué sirve el estándar FHIR?")

pprint(response.response)

('El estándar FHIR (Fast Healthcare Interoperability Resources) sirve para '
 'facilitar el intercambio electrónico de información de salud entre '
 'diferentes sistemas y organizaciones. FHIR define cómo deben estructurarse, '
 'codificarse y transmitirse los datos clínicos y administrativos, permitiendo '
 'que aplicaciones, hospitales, laboratorios y otros actores del sector salud '
 'puedan compartir información de manera segura, eficiente y estandarizada. '
 'Esto mejora la interoperabilidad, la integración de sistemas y la atención '
 'al paciente.')


Puedes inspeccionar la trazabilidad (cómo el LLM razona e intercambia mensajes)

Esta característica resulta muy útil al depurar, optimizar o auditar un sistema impulsado por IA, ya que puedes ver exactamente cómo el modelo llegó a cada respuesta.

In [3]:
dspy.inspect_history(n=1)





[2025-07-02T16:09:43.773953]

System message:

Your input fields are:
1. `question` (str):
Your output fields are:
1. `response` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `response`.


User message:

[[ ## question ## ]]
Para qué sirve el estándar FHIR?

Respond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## response ## ]]
El estándar FHIR (Fast Healthcare Interoperability Resources) sirve para facilitar el intercambio electrónico de información de salud entre diferentes sistemas y organizaciones. FHIR define cómo deben estructurarse, codificarse y transmitirse los datos clínicos y administrativos, permitiendo q

Puedes pedir razonamientos explícitos y respuestas, todo controlado. 

La metodología de "Chain of Thought" permite que el modelo exprese los pasos intermedios de su razonamiento, ayudando a obtener resultados más confiables y a descubrir posibles errores lógicos en tareas complejas. 

De esta manera, no solo obtienes la conclusión, sino la justificación detallada; esto mejora la transparencia y la confianza en sistemas IA.

In [4]:
cot = dspy.ChainOfThought('question -> response') # Define un modelo que recibe una pregunta y devuelve una respuesta, incluyendo el razonamiento detrás de la respuesta
response = cot(question="Qué recurso FHIR podría usar para representar una serie de ejercicios de rehabilitación?")
pprint(response.response)
pprint(response.reasoning)

('El recurso FHIR más adecuado para representar una serie de ejercicios de '
 'rehabilitación es el recurso "CarePlan". Este recurso permite describir un '
 'plan de atención que incluye actividades específicas, como ejercicios de '
 'rehabilitación, detallando instrucciones, frecuencia y objetivos para el '
 'paciente.')
('Para representar una serie de ejercicios de rehabilitación en FHIR, es '
 'importante identificar un recurso que permita describir actividades '
 'planificadas para un paciente, incluyendo detalles como tipo de ejercicio, '
 'frecuencia, duración y cualquier instrucción relevante. El recurso FHIR más '
 'adecuado para esto es "CarePlan", ya que está diseñado para describir un '
 'plan de atención que puede incluir actividades como ejercicios de '
 'rehabilitación. Dentro de "CarePlan", se pueden detallar las actividades '
 'específicas usando el campo "activity", y se pueden vincular recursos '
 'adicionales como "Procedure" o "ServiceRequest" si se requiere más det

Una de las fortalezas de DSPy es que no solo puedes controlar el contenido textual de las respuestas, sino también su estructura de datos. Puedes especificar que una salida debe ser booleana, una lista, diccionario, número, etc. Así se minimizan los errores al conectar con otras partes del sistema.

Esto es esencial en aplicaciones donde las decisiones automáticas requieren formatos estrictos.

In [5]:
priorizador = dspy.Predict('diagnostico: str -> es_urgente: bool') # Define un modelo que recibe un diagnóstico y retorna si es urgente o no
priorizador(diagnostico="fractura de incisivo")

Prediction(
    es_urgente=True
)

## Agente

El verdadero poder de los agentes DSPy se despliega al dotarlos de herramientas externas. Un agente puede acceder a bases de datos, llamar funciones, integrar APIs y razonar para lograr lo que se le solicita.

Supongamos una clínica con citas médicas. Definimos modelos de datos y funciones ("tools")

In [6]:
class Patient(BaseModel):
    patient_id: str
    name: str
    date_of_birth: str

class Specialty(BaseModel):
    specialty_id: str
    name: str

class Slot(BaseModel):
    slot_id: str
    specialty_id: str
    patient_id: None | str = None
    date: str
    time: str

# Agenda is a list of slots
class Agenda(BaseModel):
    slots: list[Slot]


Datos ficticios

In [7]:
patients = {
    "123": Patient(patient_id="123", name="Juan Pérez", date_of_birth="1990-01-01"),
    "456": Patient(patient_id="456", name="Ana Díaz", date_of_birth="1985-05-05"),
}

specialties = {
    "cardiolgia": Specialty(specialty_id="cardiologia", name="Cardiología"),
    "dermatologia": Specialty(specialty_id="dermatologia", name="Dermatología"),
    "neurologia": Specialty(specialty_id="neurologia", name="Neurología")
}

agenda = Agenda(slots=[
    Slot(slot_id="slot1", specialty_id="cardiologia", date="2023-10-01", time="10:00"),
    Slot(slot_id="slot2", specialty_id="dermatologia", date="2023-10-01", time="11:00"),
    Slot(slot_id="slot3", specialty_id="neurologia", date="2023-10-01", time="12:00"),
    Slot(slot_id="slot4", specialty_id="cardiologia", date="2023-10-02", time="10:00"),
    Slot(slot_id="slot5", specialty_id="dermatologia", date="2023-10-02", time="11:00"),
])

Herramientas para el agente:

Estas herramientas encapsulan funcionalidad relevante que el agente puede invocar para resolver solicitudes del usuario, manteniendo el control y la audibilidad de cada paso realizado.

In [8]:
def get_patient(patient_id: str) -> Patient | None:
    """Retrieve a patient by their ID."""
    return patients.get(patient_id)

def get_patient_id(name: str, date_of_birth: str) -> str | None:
    """Retrieve a patient ID by their name and date of birth."""
    for patient in patients.values():
        if patient.name == name and patient.date_of_birth == date_of_birth:
            return patient.patient_id
    return None

def book_slot(slot_id: str, patient_id: str) -> Slot | None:
    """Book a slot for a patient."""
    if slot_id not in [slot.slot_id for slot in agenda.slots]:
        return None
    
    for slot in agenda.slots:
        if slot.slot_id == slot_id and slot.patient_id is None:
            slot.patient_id = patient_id
            return slot
    return None

def get_specialties() -> list[Specialty]:
    """Retrieve all specialties."""
    return list(specialties.values())

def get_slots_by_specialty(specialty_id: str) -> list[Slot]:
    """Retrieve all available slots for a given specialty."""
    return [slot for slot in agenda.slots if slot.specialty_id == specialty_id and slot.patient_id is None]

Construcción del agente (ReAct agent): 

Aquí, el agente puede interpretar pedidos complejos hechos en lenguaje natural y ejecutar la secuencia de funciones necesarias para cumplirlos, manteniendo explicabilidad y control.

In [9]:
class DSPyClinicCustomerService(dspy.Signature):
    """Eres un asistente virtual para una clínica médica. Tu tarea es ayudar a los pacientes a reservar citas médicas.
    
    You are given a list of tools to handle user request, and you should decide the right tool to use in order to
    fullfil users' request."""
    user_request: str = dspy.InputField()
    process_result: str = dspy.OutputField(
        desc=(
                "Message that summarizes the process result, and the information users need, e.g., the "
                "slot_id if a new Appointment is booked."
            )
        )

In [10]:
agent = dspy.ReAct(
    DSPyClinicCustomerService,
    tools = [
        get_patient,
        get_patient_id,
        book_slot,
        get_slots_by_specialty,
        get_specialties
    ]
)

Este enfoque es ideal para automatizar flujos de trabajo complejos, reducir errores humanos y mejorar la experiencia de usuarios en entornos clínicos. 

Puedes pedir al agente otras cosas, por ejemplo listar especialidades: 

In [11]:
result = agent(user_request="Mi nombre es Juan Pérez y nací el 1 de enero de 1990. Quiero reservar una cita con el cardiólogo lo antes posible.")


In [12]:
pprint(result.process_result)
pprint(result.reasoning)
pprint(result.trajectory)

('La cita con el cardiólogo para Juan Pérez ha sido reservada exitosamente '
 'para el 1 de octubre de 2023 a las 10:00. No necesitas hacer nada más; tu '
 'cita está confirmada.')
('Primero, identifiqué al paciente Juan Pérez usando su nombre y fecha de '
 'nacimiento para obtener su ID de paciente. Luego, consulté la lista de '
 'especialidades para encontrar el ID de Cardiología. Posteriormente, busqué '
 'los horarios disponibles para esa especialidad y seleccioné el más próximo, '
 'que es el 1 de octubre de 2023 a las 10:00. Finalmente, reservé la cita para '
 'Juan Pérez en ese horario.')
{'observation_0': '123',
 'observation_1': [Specialty(specialty_id='cardiologia', name='Cardiología'),
                   Specialty(specialty_id='dermatologia', name='Dermatología'),
                   Specialty(specialty_id='neurologia', name='Neurología')],
 'observation_2': [Slot(slot_id='slot1', specialty_id='cardiologia', patient_id='123', date='2023-10-01', time='10:00'),
                

In [13]:
dspy.inspect_history(n=10)





[2025-07-02T16:09:43.773953]

System message:

Your input fields are:
1. `question` (str):
Your output fields are:
1. `response` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `response`.


User message:

[[ ## question ## ]]
Para qué sirve el estándar FHIR?

Respond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## response ## ]]
El estándar FHIR (Fast Healthcare Interoperability Resources) sirve para facilitar el intercambio electrónico de información de salud entre diferentes sistemas y organizaciones. FHIR define cómo deben estructurarse, codificarse y transmitirse los datos clínicos y administrativos, permitiendo q

In [14]:
result = agent(user_request=f"Quiero saber todas las especialidades médicas disponibles.")


In [15]:
pprint(result.process_result)
pprint(result.reasoning)
pprint(result.trajectory)

('Las especialidades médicas disponibles en la clínica son: Cardiología, '
 'Dermatología y Neurología. Si desea agendar una cita con alguna de estas '
 'especialidades, por favor indíquelo.')
('Consulté la base de datos de la clínica y obtuve la lista de especialidades '
 'médicas disponibles. Identifiqué que actualmente se ofrecen las siguientes '
 'especialidades: Cardiología, Dermatología y Neurología. Esta información es '
 'suficiente para responder a la solicitud del usuario.')
{'observation_0': [Specialty(specialty_id='cardiologia', name='Cardiología'),
                   Specialty(specialty_id='dermatologia', name='Dermatología'),
                   Specialty(specialty_id='neurologia', name='Neurología')],
 'observation_1': 'Completed.',
 'thought_0': 'El usuario quiere conocer todas las especialidades médicas '
              'disponibles. Debo consultar la lista de especialidades para '
              'proporcionarle la información solicitada.',
 'thought_1': 'Ya tengo la list

## Retrieval augmented generation

RAG es una técnica avanzada que combina las capacidades generativas del LLM con la búsqueda eficiente de información en bases de datos, documentos o corpus extensos. Esto permite a los agentes responder con base en conocimiento actual, especializado o institucional, entregando no solo una respuesta sino también evidencia de respaldo. 

Esto se traduce en respuestas más precisas, actualizadas y confiables para el usuario, evitando invenciones del modelo ("alucinaciones") al atar la generación a información documentada. 

In [16]:
ley = []
with open("/workspaces/mae2/data/ley_20584.txt", "r") as file:
    for line in file:
        ley.append(line.strip())

La búsqueda semántica se basa en comparar vectores de significado ("embeddings") en vez de texto literal, logrando encontrar los pasajes más relevantes aunque se use un lenguaje diferente al original.

In [17]:
embedder = dspy.Embedder('openai/text-embedding-3-small', dimensions=512)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=ley, k=3)

In [18]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)
rag = RAG()

Gracias a RAG, el modelo fundamenta su respuesta en la normativa vigente, mostrando no solo el qué sino el porqué basado en la evidencia extraída del corpus legal.

In [19]:
response = rag(question="Si soy un trabajador de la salud, puedo compartir la información de la ficha clínica con la prensa?")
pprint(response.response)
pprint(response.reasoning)

('No, como trabajador de la salud no puedes compartir la información de la '
 'ficha clínica con la prensa. La ley establece que la información contenida '
 'en la ficha clínica es confidencial y solo puede ser accedida por personas o '
 'instituciones expresamente autorizadas, entre las cuales no se encuentra la '
 'prensa. Difundir esta información a terceros no autorizados constituye una '
 'infracción grave a la normativa de protección de datos sensibles y a la ley '
 'sobre protección de la vida privada.')
('Según los artículos citados, la ficha clínica contiene datos sensibles y '
 'está protegida por estrictas normas de confidencialidad y acceso. El '
 'artículo 13 establece claramente que los terceros que no estén directamente '
 'relacionados con la atención de salud de la persona no tendrán acceso a la '
 'información contenida en la ficha clínica, incluyendo al personal de salud y '
 'administrativo del mismo prestador que no esté vinculado a la atención de la '
 'persona. A

In [20]:
dspy.inspect_history()





[2025-07-02T16:09:44.239689]

System message:

Your input fields are:
1. `context` (str): 
2. `question` (str):
Your output fields are:
1. `reasoning` (str): 
2. `response` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `response`.


User message:

[[ ## context ## ]]
[1] «Artículo 12.- La ficha clínica es el instrumento obligatorio en el que se registra el conjunto de antecedentes relativos a las diferentes áreas relacionadas con la salud de las personas, custodiada por uno o más prestadores de salud, en la medida que realizaron las atenciones registradas, que tiene como finalidad integrar la información necesaria en el proceso asistencial de cada persona, 